In [1]:
from util import set_seed
from argparse import Namespace
import torch
import numpy as np
from train_models_dpt import get_probs

In [4]:
config = Namespace(user='tcai',
data_dir='/shared/share_mala/implicitbayes/dataset_files/synthetic_data/bimodal/N=500,D=2500,D_eval=1000,cnts1=25,cnts2=25,forced',
                   marginal_vs_sequential='sequential',
                   batch_size=20,
                   eval_batch_size=20,
                   Z_dim=2,
                   learning_rate=0.001,
                   dataset_type='synthetic',
                   seed=2340923,
                   embed_data_dir=None,
                   extra_eval_data=None,
                   sequential_init_mean=0.5,
                   MLP_width=100,
                   MLP_layer=3,
                   repeat_suffstat=10,
                   device='cpu',
                   weight_decay=1e-2,
                   MLP_last_fn='none',
                   num_arms=10
)
data = torch.load(config.data_dir+'/eval_data.pt')

In [8]:
seed = 234234234
set_seed(seed)

In [ ]:
# select 10 arms
num_arms = 10

T = data['Y'].shape[1]
total_rows = len(data['Z'])
E = 1000 # number of environments
# assume one history per environment...
H = T # length of history

envs = []
histories = []
for e in range(E): 
    if e % 100 == 0:
        print(e)
    arm_idxs = (torch.randperm(total_rows) < num_arms).int().nonzero().flatten()

    this_Z = data['Z'][arm_idxs]
    arm_counts = torch.zeros(num_arms)
    this_Y = data['Y'][arm_idxs]
    this_best_arm = data['click_rate'][arm_idxs].argmax()
    targets = torch.zeros(num_arms)
    targets[this_best_arm] = 1

    envs.append({'Z':this_Z, 'Y':this_Y, 'best_arm':targets, 'arm_rewards': data['click_rate'][arm_idxs]})
    probs = get_probs(10)
    
    this_history = torch.zeros(H)
    for h in range(H):
        chosen_arm = np.random.choice(np.arange(num_arms), p=probs)    
        cols = torch.arange(T).unsqueeze(0)  
        this_history[h] = chosen_arm
    histories.append(this_history)

In [17]:
torch.save({'env':envs, 'hist':histories}, config.data_dir + f'/dpt_histories/eval_hist_num_arms={num_arms},E={E},H={H},S=1,seed={seed}')

In [ ]:
import os
os.mkdir(config.data_dir + '/dpt_histories')